In [ ]:
import numpy as np
import random
import statistics
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

In [ ]:
class conflict_organism:
    def __init__(self, strength):
        self.strength = strength

    def __str__(self):
        return f"{self.strength}"

    def breed(self):
        global conflict_organism_pop
        strength1 = np.round(self.strength + np.random.randn()*1.5)
        if np.random.rand()>len(conflict_organism_pop)/1000:
            conflict_organism_pop.append(conflict_organism(strength1))

In [ ]:
class organism:
    def __init__(self, strength, period, gender, family):
        self.strength = strength
        self.period = period
        self.gender = gender
        self.family=family

    def __str__(self):
            return f"{self.gender},{self.strength}({self.period})"

In [ ]:
#ranking
def reproduce():
    global m_r
    global population
    global conflict_organism_pop
    global child_population


    child_population=[]
 
    herm_pop=[p for p in population if p.gender == 'hermaphrodite'] 
    other_pop=[p for p in population if p.gender != 'hermaphrodite']
    female_pop=[p for p in population if p.gender == 'female']
    other_pop_len = len(female_pop)
    population=[]

    while len(child_population)<other_pop_len:
        
        random_org1=np.random.choice(range(len(other_pop)))
        single_parent=other_pop[random_org1]
        
        gender1=single_parent.gender

        if np.random.rand()<m_r:
            if np.random.rand()<0.5:
                gender1='hermaphrodite'
            else:
                if gender1 == 'male':
                    gender1='female'
                else:
                    gender1='male'

    
        strength1 = np.round(single_parent.strength + np.random.randn(), decimals=2)

        
        fam=single_parent.family
    
        child1 = organism(strength1, 1, gender1, fam)
        population.append(single_parent)
        other_pop.remove(other_pop[random_org1])
        child_population.append(child1)
        

    for single_parent in herm_pop:
    
        gender1='hermaphrodite'

        if np.random.rand()<m_r:
            if np.random.rand()<0.5:
                gender1='male'
            else:
                gender1='female'

    
        strength1 = np.round(single_parent.strength + np.random.randn(), decimals=2)

        
        fam=single_parent.family
    
        child1 = organism(strength1, 1, gender1, fam)
        child_population.append(child1)

    for i in other_pop:
        population.append(i)
            
    for i in herm_pop:
        population.append(i)

In [ ]:
def hunt(org):
    global conflict_organism_pop
    global population

    if len(conflict_organism_pop)>0:
        random_conflict_organism=np.random.choice(range(len(conflict_organism_pop)))
        if org.strength >= conflict_organism_pop[random_conflict_organism].strength+np.random.randn()/50:
            conflict_organism_pop.remove(conflict_organism_pop[random_conflict_organism])
            return 1
        else:
            return 0
    else:
        return 0

In [ ]:
def single_parent_hunt(org):
    global conflict_organism_pop
    global population
    global child_population

    if len(conflict_organism_pop)>0:
    
        child =[p for p in child_population if p.family == org.family]
    
        if len(child)>0:
            random_conflict_organism=np.random.choice(range(len(conflict_organism_pop)))
    
            if hunt(org) == 1:
                population.append(child[0])
                    
                if hunt(org) == 1:
                    return 1
                else:
                    return 0
            else:
                return 0
        else:
            if hunt(org) == 1:
                return 1
            else:
                return 0
    else:
        return 0

In [ ]:
def hunt_and_feed():
    global conflict_organism_pop
    global population
    global child_population


    hunting_population=population.copy()
    random.shuffle(hunting_population)
    population=[]

    for adult in hunting_population:
        if single_parent_hunt(adult) == 1:
            population.append(adult)
    
    for org in population:
        org.period+=1
    population=[p for p in population if p.period <= 4]

    random.shuffle(conflict_organism_pop)
    for conflict_org in conflict_organism_pop:
        conflict_org.breed()

    k=0
    for org in population:
        org.family=k
        k+=1

In [ ]:
share_plot=[]
population_plot=[]
number_of_iter=2000
m_r=0.05
n = 100
n_conflict_organism=300
population=[]
conflict_organism_pop=[]
share0=1

for i in range(n):
    strength=np.round(10+np.random.randn()*5)
    age=np.round(1+np.random.exponential(2))

    if age<2:
        age=2
    if age>4:
        age=4
    if np.random.rand()<m_r:
        share0+=100/n
        if np.random.rand()<0.5:
            gender='male'
        else:
            gender='female'
    else:
        gender='hermaphrodite'
        
    new_organism=organism(strength, age, gender, i)
    population.append(new_organism)

for i in range(n_conflict_organism):
    strength=np.round(12+np.random.randn()*5)
    if strength<1:
        strength=1
    new_conflict_organism=conflict_organism(strength)
    conflict_organism_pop.append(new_conflict_organism)

share_plot.append(share0)
number=0
long_term_trend=[]
long_long_term_trend=[]


while number<number_of_iter:
    reproduce()
    hunt_and_feed()
    if len(population)<=0:
        print(number, " population=0")
        break
    if len(conflict_organism_pop)<=0:
        print(number, " conflict_organism_pop=0")
        break

    str_conflict_organism=0
    for i in conflict_organism_pop:
        str_conflict_organism+=i.strength

    males=[p for p in population if p.gender == 'male']
    females=[p for p in population if p.gender == 'female']
    hermaphrodites=[p for p in population if p.gender == 'hermaphrodite']
    st_males=0

    for i in males:
        st_males+=i.strength
    st_females=0
    for i in females:
        st_females+=i.strength
    st_hermaphrodites=0
    for i in hermaphrodites:
        st_hermaphrodites+=i.strength
    
    male_share=np.round(len(males)/len(population), decimals=3)
    female_share=np.round(len(females)/len(population), decimals=3)
    share=np.round(100*(len(males)+len(females))/len(population), decimals=1)
    long_term_trend.append(share)
    long_long_term_trend.append(share)
    share_plot.append(share)
    population_plot.append(len(population)) 

    print(number+1)
    print(" ")
    print("males:", len(males))
    print("females:", len(females))
    print("hermaphrodites:", len(hermaphrodites))
    print("total:", len(population))
    print("conflict_organism:", len(conflict_organism_pop))
    print(" ")
    print("male_share:",   np.round(male_share*100,decimals=1), "%")
    print("female_share:",  np.round(female_share*100,decimals=1), "%")
    print("total_share:",  share, "%")
    if number>=10:
        del long_term_trend[0]
        print("long_term_share(over_10_periods):", np.round(statistics.mean(long_term_trend)), "%")
    if number>=500:
        del long_long_term_trend[0]
        print("very_long_term_share(over_500_periods):", np.round(statistics.mean(long_long_term_trend)), "%")
    print(" ")
    if len(males)>0:
        print("average_strength_males:", np.round(st_males/len(males)))
    if len(females)>0:
        print("average_strength_females:", np.round(st_females/len(females)))
    if len(hermaphrodites)>0:
        print("average_strength_hermaphrodites:", np.round(st_hermaphrodites/len(hermaphrodites)))
    print("average_strength_conflict_organism:", np.round(str_conflict_organism/len(conflict_organism_pop))) 
    
    print("__________________________________________________")

    number+=1

In [ ]:
fig, ax = plt.subplots(figsize=(6, 5))
    
ax.plot(share_plot[-1001:-1])
ax.set_title('Share of other hermaphrodite population\n last 1000 generations (in %)')
ax.set_xlabel('Average Share =' + str(np.round(statistics.mean(share_plot[-1001:-1]), decimals=3)), fontsize=12)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(6, 5))

ax.plot(share_plot)
ax.set_title('Share of other hermaphrodite population (in %)')
ax.set_xlabel('Average Share =' + str(np.round(statistics.mean(share_plot), decimals=3)), fontsize=12)
plt.show()